In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
import os
import sys
np.seterr(divide='ignore', all='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
def bin_feature(threshold_list, matrix):

    for i in range(len(matrix)):
        for j, value in enumerate(matrix[i].tolist()):
            if matrix[i][j] > threshold_list[j]:
                matrix[i][j] = 1
            else:
                matrix[i][j] = 0
    return matrix

def cal_feature_entropy(k, pn_list, p, n):

    sum = 0
    for i in range(k):
        p_i = pn_list[i][0]
        n_i = pn_list[i][1]
        sum += ((p_i+n_i)/(p+n)) * cal_entropy(p_i/(p_i+n_i), n_i/(p_i+n_i))
    return sum

def cal_entropy(p, n):
    a = (-p/(p+n))*np.log2(p/(p+n))
    b = (-n/(p+n))*np.log2(n/(p+n))
    if np.isnan(a):
        a = 0
    if np.isnan(b):
        b = 0

    return a+b

def Info_G(feature, y, p, n):

   
    uni_values = set([x for x in feature.tolist()])
    k = len(uni_values)
    pn_list = []
    for val in uni_values:
        p_i = 0
        n_i = 0
        for i, item in enumerate(feature):
            if item == val:
                if y[i] == 1:
                    p_i +=1
                else:
                    n_i +=1
        pn_list.append([p_i,n_i])
    return cal_entropy(p,n) - cal_feature_entropy(k, pn_list, p, n)

def find_best_feature_id(arr, y_arr, p, n, excpet_columns = [-1]):

    IGs = []
    for col_id in range(arr.shape[1]):
        if col_id not in excpet_columns:
            IGs.append([col_id, Info_G(arr[:,col_id], y_arr, p, n)])
    return sorted(IGs, key = lambda x: x[1], reverse= True)[0][0]

matrix = pd.read_csv("/Users/avanijain/Documents/ML_assignment2/spambase.data")
Data = matrix.values

np.random.seed(0)
np.random.shuffle(Data)

X = np.array(Data[:,:57])
Y = np.array(Data[:,57])
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=round(1/3,1))

m_training = np.mean(x_train, axis=0)
m_testing = np.mean(x_test, axis=0)
s_training = np.std(x_train, axis=0)
s_testing = np.std(x_test, axis=0)

#Standardizing  the data:
x_test_st = np.subtract(x_test, m_testing)
x_test_st = np.true_divide(x_test, s_testing)
x_train_st = np.subtract(x_train, m_training)
x_train_st = np.true_divide(x_train, s_training)

x_train_st = bin_feature(m_training, x_train_st)
x_test_st = bin_feature(m_testing, x_test_st)

x_train_stand_spam = []
x_train_stand_non_spam = []

for i in range(len(x_train_st)):
    # Spam
    if y_train[i] == 1:
        x_train_stand_spam.append(x_train_st[i])
    # Non-Spam
    else:
        x_train_stand_non_spam.append(x_train_st[i])
x_train_stand_spam = np.array(x_train_stand_spam)
x_train_stand_non_spam =np.array(x_train_stand_non_spam)

spam_all_means = np.mean(x_train_stand_spam, axis=0)
non_spam_all_means = np.mean(x_train_stand_non_spam, axis=0)

'''x_t = x_train_stand_spam[:2]
x_t = np.append(x_t, x_train_stand_non_spam[:1], axis=0)
y_t = [1,1,0]
y_t = np.array(y_t).reshape(3,1)'''

used_feat = []
tree = {}

def ID3(ex, spam, non_spam):
    root = {}
    counter = Counter()
    target_attr_arr =  ex[:, ex.shape[1]-1]
    for y_val in target_attr_arr:
        counter[y_val] +=1
    if (len(counter) == 1):
        root["label"] = counter.most_common(1)[0][0]
        return root
    if len(used_feat) == ex.shape[1]-1:
        root["label"] = counter.most_common(1)[0][0]
        return root
    else:
        # A ← The Attribute that best classifies examples.
        ex_without_target = ex[:, :ex.shape[1]-1]
        best_feat_id = find_best_feature_id(ex_without_target, target_attr_arr, spam,non_spam, used_feat)
        used_feat.append(best_feat_id)
        # Decision Tree attribute for Root = A.
        root["feature"] = best_feat_id
        root["sub_nodes"] = {}
        x_values = set()
        x_values.update(ex[:, best_feat_id])
        
        for x_value in x_values:
            counter = Counter()
            tmp_data = ex[ex[:, best_feat_id] == x_value]
            # If Examples(vi) is empty
            if len(tmp_data) == 0:
                for y_val in tmp_data[:, tmp_data.shape[1] - 1]:
                    counter[y_val] += 1
                root["sub_nodes"][x_value] = {"leaf": counter.most_common(1)[0][0]}
            else:
                root["sub_nodes"][x_value] = ID3(tmp_data, spam, non_spam)
        return root
    

y_pred_list = []
def pred(tree, x_arr, iter):

    if "label" in tree:
        y_pred_list.append(tree["label"])
        return y_pred_list
    elif 'feature' in tree:
        sub_nodes = tree["sub_nodes"]
        root_feat_id = tree["feature"]
        pred(tree["sub_nodes"][x_arr[root_feat_id]], x_arr, iter)
        
tree = ID3(np.append(x_train_st, np.array(y_train).reshape(len(y_train),1), axis=1), len(x_train_stand_spam), len(x_train_stand_non_spam))

accuracy = 0
TP = 0
FP = 0
FN = 0
TN = 0

for i, sample in enumerate(x_test_st.tolist()):
    if not i in [600, 616, 840, 1034, 1167]:
        #print("i:", i)
        y_pred_list.clear()
        pred(tree, sample, i)
        prediction = y_pred_list[0]
        if y_test[i] == prediction:
            accuracy += 1
        if y_test[i] == prediction and y_test[i] == 1:
            TP += 1
        elif y_test[i] != prediction and prediction == 1:
            FP += 1
        elif y_test[i] != prediction and prediction == 0:
            FN += 1
precision = TP/(TP+FP)
recall = TP/(TP+FN)
f_measure = (2*precision*recall)/(precision+recall)
accuracy = accuracy/(len(y_test)-10) * 100

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F-Measure:", f_measure)

Accuracy: 84.23357664233576
Precision: 0.8330097087378641
Recall: 0.7606382978723404
F-Measure: 0.7951807228915663
